In [1]:
from datetime import datetime
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd

In [2]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

In [3]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
)

In [4]:
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=10000,
  where=[
    snapshot.Proposal.space == 'olympusdao.eth',
    snapshot.Proposal.state == 'closed',
  ]
)

In [5]:
proposals_snapshots = sg.query_df([
  proposals.title,
  proposals.id,
  proposals.body
])

In [6]:
proposals_choices = sg.query(proposals.choices)

TypeError: 'FieldPath' object is not iterable

In [85]:
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_a', 'option_b', 'option_c', 'option_d', 'option_e','option_f','option_g'])

In [86]:
olympus_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)

In [87]:
##let's view the output just to make sure
olympus_governance_view

,proposals_title,proposals_id,proposals_body,option_a,option_b,option_c,option_d,option_e,option_f,option_g
0,OIP-94: Interim Ranged Stability Policy Levers,0x6413984c8f1076e3f46f196810ba51781f25f4d2f1eb...,abipup\n \n 2 days ago\n\nSumma...,Inverse bonds & adjust liquidity,Do nothing,None,None,None,None,None
1,OIP-93: Mint and Sync,0xf0e62d0a9eeb47a295bab9a8d0cc84c0d4de0a8cb000...,Summary:\n\nEnable minting into liquidity pool...,"1. Implement, no change to RR","2. Implement, change RR",3. Do not implement,None,None,None,None
2,TAP-12 Pilot Deposit on Tribe Turbo,0x7d191f4a7f2ea3d138c07f73d568b93c96fc5ffce267...,Summary\n\nSeek authority from the community t...,Approve Pilot Deposit,Do not approve pilot deposit,None,None,None,None,None
3,TAP-11 Ribbon Finance Whitelist,0xe52a4af6307bea40a34f2eaf9a93f11c8abf382020cc...,Summary: Seek authority from the community to ...,Allow Ribbon Finance,Do not allow Ribbon Finance,None,None,None,None,None
4,OIP - 92 : Whitelist FEI as a treasury Reserve...,0xe9aa47f39bc5ace6ad5a19c08007a3683386e19c1390...,Summary\n\nWhitelist FEI as our next reserve a...,Approve FEI in the Treasury,Do not approve FEI in Treasury,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
192,OIP-5: Lower LP Staking Rewards,QmTrwGU99RMqTeeCKy5oW9Le9M2mXHHdNeEWzn7T67watz,Summary: We should decrease LP staking rewards...,For: Option 1,For: Option 2,Against,None,None,None,None
193,Deposit SLP,QmZhSNWAzrDy5KiGSeBsKBXUHW6D4vJnfCuqmdai8LPETd,Summary: The DAO currently holds liquidity tha...,Deposit; keep Onsen,Deposit; no Onsen,Don't deposit,None,None,None,None
194,Loosen Policy,QmZJb2evEjsxtLz92XBuhApWty69fVqUpFHzuVjMjCd3Se,Summary: Let's adjust system parameters away f...,Do it,Don't do it,None,None,None,None,None
195,Add DAI Bonds,QmUC5xa4YaWZBV7fy241svBQ8bHSMksm8kB7avk1WyWv9m,Summary: We can launch bonds using DAI instead...,Add; remove sales,Add; phase out sales,Add; keep sales,Do not add,None,None,None


In [88]:
total_proposals = len(olympus_governance_view)
total_proposals

197

In [89]:
   proposal_id = olympus_governance_view.iloc[0,1]

In [90]:
print(proposal_id)

0x6413984c8f1076e3f46f196810ba51781f25f4d2f1ebee606455929cbbc7d7c8


In [91]:

    vote_tracker = snapshot.Query.votes(
    orderBy = 'created',
    orderDirection='desc',
    first=10000,
    where=[
      snapshot.Vote.proposal == proposal_id
    ]
    )

In [92]:
voting_snapshots_list = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice
])

In [93]:
voting_snapshots_list

,votes_id,votes_voter,votes_created,votes_choice
0,0x5730304152f17584521699d09b7316a1f9adaf105a03...,0xf484a6a2cCe70DFDf58E2029B4eC9c45a16dfd2B,1650931003,1
1,0x60efbe2fe72a57c8a07b4f8cbb8bec4c4e22268785d0...,0x9a2Fe6b12EAA8EE9e95Da5BE625851a9A6836662,1650930857,1
2,0x9b3179d0f083281e6b4bd097a189c0d7d5cbb8136a3e...,0x8F633B9ED467ff943f82B1cCC98EAbE12d33Ed1a,1650930302,1
3,0x5c928772509d6f3571607e2243265221a5cc2c5845ec...,0x3DbaebCc4C49408b187Ae49686e4df46978206Dd,1650930257,1
4,0x87e0ae40c7e4d918d491dc4446a634d78c1d98350022...,0xf1B98B42D482B86fF88B3546A5BAFF867337c845,1650929362,1
...,...,...,...,...
352,0xebf8606b82ef893c59132a3a08f734b26cde4f7e46a0...,0xC2c764F02b838DbFbCc01aa5280Fcd7D906A02E5,1650651713,1
353,0xcf79114e54001252e22e40124bf5becdd74fcfd0b8a2...,0xf10918Fc1725A65960db2b906b4868a0Fdbb6870,1650650875,1
354,0x6791123892e904ce33c96828d01849330a757bbefc11...,0x6487d336bDec94d4714a4d217B15aa5c7b9dfA99,1650649071,1
355,0x553c6c34caabe52eb76cd1148ec6bc8f7b28574b5357...,0xa100cD30f2384E53d149729Db33a84597c296c3d,1650648931,1


In [94]:
voting_snapshots_list['Proposal'] = proposal_id
print(voting_snapshots_list)

                                              votes_id  \
0    0x5730304152f17584521699d09b7316a1f9adaf105a03...   
1    0x60efbe2fe72a57c8a07b4f8cbb8bec4c4e22268785d0...   
2    0x9b3179d0f083281e6b4bd097a189c0d7d5cbb8136a3e...   
3    0x5c928772509d6f3571607e2243265221a5cc2c5845ec...   
4    0x87e0ae40c7e4d918d491dc4446a634d78c1d98350022...   
..                                                 ...   
352  0xebf8606b82ef893c59132a3a08f734b26cde4f7e46a0...   
353  0xcf79114e54001252e22e40124bf5becdd74fcfd0b8a2...   
354  0x6791123892e904ce33c96828d01849330a757bbefc11...   
355  0x553c6c34caabe52eb76cd1148ec6bc8f7b28574b5357...   
356  0x4c1841cedc1fd0eaede37aaee23114b7796cd700089a...   

                                    votes_voter  votes_created  votes_choice  \
0    0xf484a6a2cCe70DFDf58E2029B4eC9c45a16dfd2B     1650931003             1   
1    0x9a2Fe6b12EAA8EE9e95Da5BE625851a9A6836662     1650930857             1   
2    0x8F633B9ED467ff943f82B1cCC98EAbE12d33Ed1a     1650930302 

In [95]:
x=0
while x <total_proposals:
##print(olympus_governance_view.iloc[2,1])
    proposal_id = olympus_governance_view.iloc[x,1]

    vote_tracker = snapshot.Query.votes(
    orderBy = 'created',
    orderDirection='desc',
    first=10000,
    where=[
      snapshot.Vote.proposal == proposal_id
    ]
    )
    voting_snapshots = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice
    ])
    voting_snapshots['Proposal'] = proposal_id
    voting_snapshots_list=pd.concat([voting_snapshots_list, voting_snapshots])
    ##voting_snapshots['Proposal'] = proposal_id

#    pd.concat([voting_snapshots_list, voting_snapshots])
    x=x+1

In [96]:
len(voting_snapshots_list)

44670

In [97]:
    path = '/Users/[user]]/Downloads/OIPS/votes_table.csv'
    voting_snapshots_list.to_csv(path, index = False)

OSError: Cannot save file into a non-existent directory: '/Users/[user]]/Downloads/OIPS'

In [1]:
from web3 import Web3, EthereumTesterProvider
w3 = Web3(EthereumTesterProvider())
w3.isConnected()

ModuleNotFoundError: No module named 'eth_tester'

In [4]:
w3 = Web3(Web3.IPCProvider('./path/to/geth.ipc'))
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3 = Web3(Web3.WebsocketProvider('ws://127.0.0.1:8546'))

In [5]:
w3.isConnected()

False